# Education CrewAI Multi-Agent System

This notebook demonstrates a multi-agent system for the education industry using CrewAI. The system automates curriculum development, personalized learning, assessment creation, and research.

You can provide a subject name and context as input, and the agents will collaborate to generate a curriculum, personalized learning paths, and assessments.

In [1]:
%pip install crewai[tools]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.9/583.9 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.2/300.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.8 MB/s eta 0:0

## Imports and Setup

Import required modules and set up any necessary environment variables.

In [3]:
from crewai import Agent, Task, Crew
from crewai import LLM
from crewai_tools import ScrapeWebsiteTool # Custom or CrewAI-provided tool

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


## User Input

Provide the subject name and context for the educational workflow.

In [4]:
subject = input("Enter the subject name (e.g., 'Artificial Intelligence'): ")
context = input("Enter the context or focus area (e.g., 'For high school students, focus on fundamentals and applications'): ")

Enter the subject name (e.g., 'Artificial Intelligence'): AI
Enter the context or focus area (e.g., 'For high school students, focus on fundamentals and applications'): basic of ai for school  children


# LLM Initialization

The LLM (Language Model) is a core component that powers our assistant's ability to generate dynamic, context-aware responses. In our setup, we initialize the LLM using specific parameters such as the model name, temperature, API base URL, and API key.

### Key Parameters:
- **model**: Specifies which language model to use (e.g., "gpt-3.5-turbo"). This determines the capabilities and performance of the generated responses.
- **temperature**: Controls the randomness of the output. A higher temperature (e.g., 0.7) leads to more diverse and creative responses, while a lower value produces more deterministic outputs.
- **base_url**: The endpoint URL for the API. This is where requests are sent.
- **api_key**: A unique key used to authenticate your requests to the API.



In [5]:

llm_agent = LLM(
    model="gpt-3.5-turbo",
    temperature=0.7,
    base_url="https://api.nexus.navigatelabsai.com",
    api_key="sk-3yaE7AQUcjAqByOQ_TIx2A"
)

##Define the tool

In [7]:
# To enable scrapping any website it finds during it's execution
scrape_tool = ScrapeWebsiteTool()


## Define Agents

Each agent has a specific role and uses appropriate tools.

In [11]:
researcher_agent = Agent(
    role='Research Specialist',
    goal='Research and gather updated information, methods, and resources',
    backstory='An academic researcher skilled at finding latest innovations in education and assessment methods.',
    tools=[scrape_tool],
    llm=llm_agent,
    allow_delegation=True,
)

curriculum_developer_agent = Agent(
    role='Curriculum Developer',
    goal='Design a comprehensive and updated curriculum based on research insights',
    backstory='An experienced curriculum strategist who can design effective educational roadmaps.',
    #tools=[QueryTool()],
    llm=llm_agent,
    allow_delegation=True
)

personalized_learning_agent = Agent(
    role='Personalized Learning Architect',
    goal='Create personalized learning plans for students with different learning styles and abilities',
    backstory='An adaptive learning expert passionate about making education personalized and inclusive.',
    #tools=[QueryTool()],
    llm=llm_agent,
    allow_delegation=True
)

assessment_creator_agent = Agent(
    role='Assessment Designer',
    goal='Develop assessments, quizzes, and evaluation tools aligned to the personalized curriculum',
    backstory='An expert in educational measurement and assessment creation.',
    #tools=[QueryTool()],
    llm=llm_agent,
    allow_delegation=True
)

## Define Tasks

Each task is linked to an agent and uses the subject/context as input.

In [13]:
research_task = Task(
    description=f"Research the latest trends, methods, and best practices in curriculum development, personalized learning, and assessment creation for {subject}. Context: {context}",
    agent=researcher_agent,
    expected_output="A detailed research report summarizing latest trends, methods, best practices, and resources for curriculum development, personalized learning, and assessment."
)

curriculum_task = Task(
    description=f"Using research findings, develop a structured curriculum outline for {subject}. Context: {context}",
    agent=curriculum_developer_agent,
    expected_output="A structured curriculum document that includes course modules, topics, learning objectives, timeline, and suggested resources."
)

personalized_learning_task = Task(
    description=f"Design multiple personalized learning paths for {subject} based on the structured curriculum for diverse learner profiles. Context: {context}",
    agent=personalized_learning_agent,
    expected_output="Personalized learning plans for at least three learner profiles: visual learner, auditory learner, and kinesthetic learner. Each plan should include recommended activities and study methods."
)

assessment_task = Task(
    description=f"Create detailed assessments including quizzes, tests, and assignments for {subject} based on the personalized learning paths. Context: {context}",
    agent=assessment_creator_agent,
    expected_output="A set of assessments including a quiz (10 MCQs), a short answer test, and a practical assignment/project, aligned with the curriculum and personalized learning paths."
)


## Setup the Crew

Combine agents and tasks into a Crew for collaborative execution.

In [14]:
education_crew = Crew(
    agents=[
        researcher_agent,
        curriculum_developer_agent,
        personalized_learning_agent,
        assessment_creator_agent
    ],
    tasks=[
        research_task,
        curriculum_task,
        personalized_learning_task,
        assessment_task
    ],
    verbose=True
)

## Run the Crew Workflow

Execute the workflow and display the final output.

In [ ]:
result = education_crew.kickoff()
print("### FINAL OUTPUT ###")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1b30a2fb-8cf6-4d6f-a6e3-69f8842a2253                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: a3ad3ce1-b2b6-46f5-a330-e886ff7adc21
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: a3ad3ce1-b2b6-46f5-a330-e886ff7adc21
       Status: Executing Task...
    └── 🤖 Agent: Research Specialist
            Status: In Progress

# Agent: Research Specialist
## Task: Research the latest trends, methods, and best practices in curriculum development, personalized learning, and assessment creation for AI. Context: basic of ai for school  children


🤖 Agent: Research Specialist
    Status: In Progress
└── 🧠 Thinking...

...

🚀 Crew: crew
└── 📋 Task: a3ad3ce1-b2b6-46f5-a330-e886ff7adc21
       Assigned to: Research Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Research Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a3ad3ce1-b2b6-46f5-a330-e886ff7adc21                                                                     │
│  Agent: Research Specialist                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
├── 📋 Task: a3ad3ce1-b2b6-46f5-a330-e886ff7adc21
│      Assigned to: Research Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Specialist
│           Status: ✅ Completed
└── 📋 Task: 9c5c9f84-cbc4-4e73-b574-132f0f0895a1
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: a3ad3ce1-b2b6-46f5-a330-e886ff7adc21
│      Assigned to: Research Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Specialist
│           Status: ✅ Completed
└── 📋 Task: 9c5c9f84-cbc4-4e73-b574-132f0f0895a1
       Status: Executing Task...
    └── 🤖 Agent: Curriculum Developer
            Status: In Progress

# Agent: Curriculum Developer
## Task: Using research findings, develop a structured curriculum outline for AI. Context: basic of ai for school  children
